![Logo BV IBMEC](https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/logo-bv-ibmec-notebooks.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ian-iania/IBMEC-BV-Modelos-Preditivos/blob/main/notebooks/NB01_Regression.ipynb)

# NB01 - Regressão e Séries Temporais (FP&A Banco BV)

**Objetivo:** aplicar modelos de regressão e de séries temporais em dados sintéticos de FP&A do BV.
**Você vai comparar baseline vs modelos, usando split temporal e métricas simples (MAE/RMSE).**
**Foco em intuição de negócio: escolher modelo útil e robusto para apoiar decisão.**


## 1) Setup do notebook

### O que vamos fazer
Importar bibliotecas e configurar funções utilitárias para reaproveitar o pipeline.

### Por que isso importa para FP&A
Padronizar o processo evita erros e facilita comparar modelos com critério.

### O que você deve ver no output
Pouca saída visual. Sem erro = ambiente pronto.


In [ ]:
# RUN_ME 1 - imports principais
# Tabelas e operações numéricas
import pandas as pd
import numpy as np

# Visualização (sem seaborn, alinhado à aula)
import matplotlib.pyplot as plt

# Modelos de regressão
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Métricas
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Modelos de séries temporais
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Utilitários de sistema
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# RUN_ME 2 - função para carregar dataset com fallback
# Ordem de tentativa:
# 1) caminho relativo ../data/
# 2) URL raw do GitHub
# 3) upload manual no Colab

def load_dataset(file_name: str) -> pd.DataFrame | None:
    local_path = Path('../data') / file_name
    if local_path.exists():
        print(f'Carregando local: {local_path}')
        return pd.read_csv(local_path)

    raw_url = f'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/{file_name}'
    try:
        print(f'Carregando via GitHub raw: {raw_url}')
        return pd.read_csv(raw_url)
    except Exception as e:
        print('Nao foi possivel carregar local/URL.')
        print(f'Erro: {e}')
        print('Use a celula de upload manual abaixo e rode novamente.')
        return None


In [ ]:
# RUN_ME 3 (opcional) - upload manual de CSV no Colab
# Use esta celula apenas se local/URL falharem.
try:
    from google.colab import files
    uploaded = files.upload()
    print('Arquivos enviados:', list(uploaded.keys()))
except Exception as e:
    print('Upload manual disponivel apenas no Google Colab.')
    print('Detalhe:', e)


In [ ]:
# RUN_ME 4 - utilitários de split, métricas e treino
# Split temporal (sem embaralhar): treino no passado, teste no futuro

def temporal_split(df: pd.DataFrame, test_size: float = 0.2):
    split_idx = int(len(df) * (1 - test_size))
    train_df = df.iloc[:split_idx].copy()
    test_df = df.iloc[split_idx:].copy()
    return train_df, test_df

# Métricas simples para comparar modelos

def calc_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

# Treina e avalia uma lista de modelos de regressão

def train_predict_evaluate(models_dict, X_train, y_train, X_test, y_test, baseline_value):
    results = []
    preds = {}
    fitted = {}

    # Baseline naive: repete ultimo valor observado do treino
    y_pred_baseline = np.repeat(baseline_value, len(y_test))
    mae_b, rmse_b = calc_metrics(y_test, y_pred_baseline)
    results.append({'modelo': 'Baseline_Naive', 'mae': mae_b, 'rmse': rmse_b})
    preds['Baseline_Naive'] = y_pred_baseline

    # Modelos de ML
    for name, model in models_dict.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae, rmse = calc_metrics(y_test, y_pred)
        results.append({'modelo': name, 'mae': mae, 'rmse': rmse})
        preds[name] = y_pred
        fitted[name] = model

    results_df = pd.DataFrame(results).sort_values('rmse', ascending=True).reset_index(drop=True)
    return results_df, preds, fitted


## 2) PARTE A - Regressão com dataset linear (X -> y)

### O que vamos fazer
Treinar e comparar vários modelos no dataset linear.

### Por que isso importa para FP&A
Ajuda a entender quais drivers explicam melhor a originação e qual modelo tem melhor erro no futuro.

### O que você deve ver no output
Tabela de métricas (MAE/RMSE), gráficos de previsão e importâncias de variáveis.


In [ ]:
# RUN_ME 5 - carregar dataset linear
df_lin = load_dataset('bv_fpa_regressao_linear.csv')
if df_lin is not None:
    # Converte data e ordena para manter consistência temporal
    df_lin['ds'] = pd.to_datetime(df_lin['ds'])
    df_lin = df_lin.sort_values('ds').reset_index(drop=True)

df_lin.head() if df_lin is not None else None


In [ ]:
# RUN_ME 6 - inspeção rápida (head, shape, dtypes)
if df_lin is not None:
    print('shape:', df_lin.shape)
    print('
Tipos de coluna:')
    print(df_lin.dtypes)


### 2.1 Visualizações rápidas (exploratórias)

### O que vamos fazer
Ver comportamento de `y` no tempo e relação simples entre `selic` e `y`.

### Por que isso importa para FP&A
Antes de modelar, é essencial enxergar tendência e possíveis relações entre variáveis.

### O que você deve ver no output
Um gráfico de linha e um gráfico de dispersão.


In [ ]:
# RUN_ME 7 - linha de y no tempo
if df_lin is not None:
    plt.figure(figsize=(10, 4))
    plt.plot(df_lin['ds'], df_lin['y'], color='#1f77b4', linewidth=2)
    plt.title('Dataset linear - Originação (y) ao longo do tempo')
    plt.xlabel('Data')
    plt.ylabel('y (R$ milhões)')
    plt.grid(alpha=0.3)
    plt.show()


In [ ]:
# RUN_ME 8 - scatter selic vs y
if df_lin is not None:
    plt.figure(figsize=(6, 4))
    plt.scatter(df_lin['selic'], df_lin['y'], alpha=0.75, color='#ff7f0e')
    plt.title('Dataset linear - SELIC vs y')
    plt.xlabel('SELIC')
    plt.ylabel('y (R$ milhões)')
    plt.grid(alpha=0.3)
    plt.show()


### 2.2 Preparar X/y e split temporal

### O que vamos fazer
Separar variáveis explicativas (`X`) e alvo (`y`), depois dividir treino/teste no tempo (80/20).

### Por que isso importa para FP&A
Em forecasting, o futuro não pode “vazar” para o treino.

### O que você deve ver no output
Tamanhos de treino e teste.


In [ ]:
# RUN_ME 9 - seleção de features e split temporal
feature_cols = ['selic', 'desemprego', 'ltv_medio', 'spread', 'marketing', 'mix_auto', 'mes']

a_train, a_test = temporal_split(df_lin, test_size=0.2)

X_train_a = a_train[feature_cols]
X_test_a = a_test[feature_cols]
y_train_a = a_train['y']
y_test_a = a_test['y']

print('Treino:', X_train_a.shape, 'Teste:', X_test_a.shape)
print('Periodo treino:', a_train['ds'].min().date(), '->', a_train['ds'].max().date())
print('Periodo teste :', a_test['ds'].min().date(), '->', a_test['ds'].max().date())


### 2.3 Definir baseline e modelos

### O que vamos fazer
Comparar baseline naive e seis modelos de ML.

### Por que isso importa para FP&A
Se um modelo não supera baseline, ele não agrega valor de negócio.

### O que você deve ver no output
Lista de modelos e valor baseline.


In [ ]:
# RUN_ME 10 - baseline e dicionário de modelos
# Baseline naive: ultimo y do treino
baseline_a = y_train_a.iloc[-1]
print('Baseline (ultimo valor do treino):', round(float(baseline_a), 2))

# Modelos (reprodutibilidade com random_state)
models = {
    'LinearRegression': Pipeline([
        ('scaler', StandardScaler()),
        ('model', LinearRegression())
    ]),
    'Ridge': Pipeline([
        ('scaler', StandardScaler()),
        ('model', Ridge(alpha=1.0, random_state=42))
    ]),
    'ElasticNet': Pipeline([
        ('scaler', StandardScaler()),
        ('model', ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000))
    ]),
    'DecisionTree': DecisionTreeRegressor(max_depth=4, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

print('Modelos prontos:', list(models.keys()))


### 2.4 Treinar e avaliar no dataset linear

### O que vamos fazer
Treinar todos os modelos e comparar MAE/RMSE no teste.

### Por que isso importa para FP&A
Decisão de modelo deve ser baseada em erro fora da amostra (teste).

### O que você deve ver no output
Tabela ordenada por RMSE.


In [ ]:
# RUN_ME 11 - treino e avaliação (PARTE A)
results_a, preds_a, fitted_a = train_predict_evaluate(
    models_dict=models,
    X_train=X_train_a,
    y_train=y_train_a,
    X_test=X_test_a,
    y_test=y_test_a,
    baseline_value=baseline_a
)

results_a


In [ ]:
# RUN_ME 12 - real vs previsto (baseline + 2 melhores)
# Seleciona melhor linear e melhor geral para comparar com baseline
linear_family = ['LinearRegression', 'Ridge', 'ElasticNet']

best_linear_a = results_a[results_a['modelo'].isin(linear_family)].iloc[0]['modelo']
best_overall_a = results_a[~results_a['modelo'].eq('Baseline_Naive')].iloc[0]['modelo']

to_plot = ['Baseline_Naive', best_linear_a, best_overall_a]
to_plot = list(dict.fromkeys(to_plot))

plt.figure(figsize=(11, 4))
plt.plot(a_test['ds'], y_test_a.values, label='Real', color='black', linewidth=2)
for name in to_plot:
    plt.plot(a_test['ds'], preds_a[name], label=name, linewidth=1.8)

plt.title('PARTE A - Real vs Previsto (teste)')
plt.xlabel('Data')
plt.ylabel('y (R$ milhões)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print('Melhor linear:', best_linear_a)
print('Melhor geral :', best_overall_a)


### 2.5 Feature importance (Random Forest e GBM)

### O que vamos fazer
Visualizar importância de variáveis em modelos de árvore.

### Por que isso importa para FP&A
Ajuda a priorizar drivers para discussão de negócio.

### O que você deve ver no output
Dois gráficos de barras de importância.


In [ ]:
# RUN_ME 13 - importância de variáveis (RandomForest)
rf = fitted_a['RandomForest']
rf_imp = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=True)

plt.figure(figsize=(7, 4))
plt.barh(rf_imp.index, rf_imp.values, color='#2ca02c')
plt.title('PARTE A - Feature importance (RandomForest)')
plt.xlabel('Importância relativa')
plt.grid(axis='x', alpha=0.3)
plt.show()


In [ ]:
# RUN_ME 14 - importância de variáveis (GradientBoosting)
gbm = fitted_a['GradientBoosting']
gbm_imp = pd.Series(gbm.feature_importances_, index=feature_cols).sort_values(ascending=True)

plt.figure(figsize=(7, 4))
plt.barh(gbm_imp.index, gbm_imp.values, color='#9467bd')
plt.title('PARTE A - Feature importance (GradientBoosting)')
plt.xlabel('Importância relativa')
plt.grid(axis='x', alpha=0.3)
plt.show()


**Nota importante:** importância de variável **não é causalidade**. Em dados com drivers correlacionados (ex.: `selic` e `spread`), interpretação deve ser feita com cuidado.


## 3) PARTE B - Mesmo pipeline no dataset não-linear

### O que vamos fazer
Repetir o pipeline no dataset com limiares/interações não-lineares.

### Por que isso importa para FP&A
Mostra na prática quando modelos não-lineares tendem a ganhar.

### O que você deve ver no output
Tabela de métricas e gráfico comparando melhor linear vs melhor não-linear.


In [ ]:
# RUN_ME 15 - carregar dataset não-linear
df_nlin = load_dataset('bv_fpa_regressao_nonlinear.csv')
if df_nlin is not None:
    df_nlin['ds'] = pd.to_datetime(df_nlin['ds'])
    df_nlin = df_nlin.sort_values('ds').reset_index(drop=True)

df_nlin.head() if df_nlin is not None else None


In [ ]:
# RUN_ME 16 - split temporal e preparação de X/y (PARTE B)
b_train, b_test = temporal_split(df_nlin, test_size=0.2)

X_train_b = b_train[feature_cols]
X_test_b = b_test[feature_cols]
y_train_b = b_train['y']
y_test_b = b_test['y']

baseline_b = y_train_b.iloc[-1]
print('Treino:', X_train_b.shape, 'Teste:', X_test_b.shape)
print('Baseline B:', round(float(baseline_b), 2))


In [ ]:
# RUN_ME 17 - treino e métricas (PARTE B)
results_b, preds_b, fitted_b = train_predict_evaluate(
    models_dict=models,
    X_train=X_train_b,
    y_train=y_train_b,
    X_test=X_test_b,
    y_test=y_test_b,
    baseline_value=baseline_b
)

results_b


In [ ]:
# RUN_ME 18 - melhor linear vs melhor não-linear (PARTE B)
linear_family = ['LinearRegression', 'Ridge', 'ElasticNet']
nonlinear_family = ['DecisionTree', 'RandomForest', 'GradientBoosting']

best_linear_b = results_b[results_b['modelo'].isin(linear_family)].iloc[0]['modelo']
best_nonlinear_b = results_b[results_b['modelo'].isin(nonlinear_family)].iloc[0]['modelo']

plt.figure(figsize=(11, 4))
plt.plot(b_test['ds'], y_test_b.values, label='Real', color='black', linewidth=2)
plt.plot(b_test['ds'], preds_b[best_linear_b], label=f'Linear: {best_linear_b}', linewidth=1.8)
plt.plot(b_test['ds'], preds_b[best_nonlinear_b], label=f'Nao-linear: {best_nonlinear_b}', linewidth=1.8)

plt.title('PARTE B - Melhor linear vs melhor não-linear (teste)')
plt.xlabel('Data')
plt.ylabel('y (R$ milhões)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print('Melhor linear B    :', best_linear_b)
print('Melhor não-linear B:', best_nonlinear_b)


**Leitura didática da PARTE B:** quando existem limiares, interações e não-linearidade, modelos de árvore/boosting costumam capturar melhor esses padrões.


## 4) PARTE C - Séries temporais (t -> y)

### O que vamos fazer
Comparar Baseline, ETS, ARIMA, ARIMAX e Prophet (opcional) no dataset temporal.

### Por que isso importa para FP&A
Nem sempre teremos bons drivers explicativos; às vezes o próprio histórico de `y` já contém sinal forte.

### O que você deve ver no output
Tabela de MAE/RMSE no teste + conclusão de uso X->y vs t->y.


In [ ]:
# RUN_ME 19 - carregar dataset de séries temporais
df_ts = load_dataset('bv_fpa_timeseries.csv')
if df_ts is not None:
    df_ts['ds'] = pd.to_datetime(df_ts['ds'])
    df_ts = df_ts.sort_values('ds').reset_index(drop=True)

df_ts.head() if df_ts is not None else None


In [ ]:
# RUN_ME 20 - plot da série y no tempo
if df_ts is not None:
    plt.figure(figsize=(10, 4))
    plt.plot(df_ts['ds'], df_ts['y'], color='#1f77b4', linewidth=2)
    plt.title('PARTE C - Série temporal de y')
    plt.xlabel('Data')
    plt.ylabel('y (R$ milhões)')
    plt.grid(alpha=0.3)
    plt.show()


In [ ]:
# RUN_ME 21 - split temporal holdout final (12 meses)
holdout = 12
train_ts = df_ts.iloc[:-holdout].copy()
test_ts = df_ts.iloc[-holdout:].copy()

print('Treino TS:', train_ts.shape, 'Teste TS:', test_ts.shape)
print('Periodo treino:', train_ts['ds'].min().date(), '->', train_ts['ds'].max().date())
print('Periodo teste :', test_ts['ds'].min().date(), '->', test_ts['ds'].max().date())


In [ ]:
# RUN_ME 22 - baseline naive (ultimo valor observado)
y_train_ts = train_ts['y'].values
y_test_ts = test_ts['y'].values

pred_naive_ts = np.repeat(y_train_ts[-1], len(y_test_ts))
mae_naive_ts, rmse_naive_ts = calc_metrics(y_test_ts, pred_naive_ts)

print('Baseline Naive - MAE :', round(mae_naive_ts, 3))
print('Baseline Naive - RMSE:', round(rmse_naive_ts, 3))


### 4.1 ETS / Holt-Winters

### O que vamos fazer
Ajustar modelo com nível, tendência e sazonalidade.

### Por que isso importa para FP&A
Séries mensais frequentemente têm sazonalidade anual.

### O que você deve ver no output
Predição para o período de teste e métricas.


In [ ]:
# RUN_ME 23 - ETS/Holt-Winters
ets_model = ExponentialSmoothing(
    train_ts['y'],
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit(optimized=True)

pred_ets = ets_model.forecast(len(test_ts)).values
mae_ets, rmse_ets = calc_metrics(y_test_ts, pred_ets)

print('ETS - MAE :', round(mae_ets, 3))
print('ETS - RMSE:', round(rmse_ets, 3))


### 4.2 ARIMA e ARIMAX

### O que vamos fazer
Treinar ARIMA puro e ARIMAX com `selic` como variável exógena.

### Por que isso importa para FP&A
ARIMAX permite incorporar driver externo sem perder estrutura temporal.

### O que você deve ver no output
Métricas de ARIMA e ARIMAX.


In [ ]:
# RUN_ME 24 - ARIMA (sem exógena)
arima_model = SARIMAX(
    train_ts['y'],
    order=(1, 1, 1),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

pred_arima = arima_model.forecast(steps=len(test_ts)).values
mae_arima, rmse_arima = calc_metrics(y_test_ts, pred_arima)

print('ARIMA - MAE :', round(mae_arima, 3))
print('ARIMA - RMSE:', round(rmse_arima, 3))


In [ ]:
# RUN_ME 25 - ARIMAX (com selic exógena)
arimax_model = SARIMAX(
    train_ts['y'],
    exog=train_ts[['selic']],
    order=(1, 1, 1),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

pred_arimax = arimax_model.forecast(steps=len(test_ts), exog=test_ts[['selic']]).values
mae_arimax, rmse_arimax = calc_metrics(y_test_ts, pred_arimax)

print('ARIMAX - MAE :', round(mae_arimax, 3))
print('ARIMAX - RMSE:', round(rmse_arimax, 3))


### 4.3 Prophet (opcional)

### O que vamos fazer
Instalar e testar Prophet com regressoras opcionais (`selic`, `evento`).

### Por que isso importa para FP&A
É um modelo popular para forecast com sazonalidade e efeitos extras.

### O que você deve ver no output
Se disponível, métricas do Prophet. Se não, instruções claras.


In [ ]:
# RUN_ME 26 (opcional) - instalar/importar Prophet
import importlib
import subprocess
import sys

prophet_ok = False

try:
    from prophet import Prophet
    prophet_ok = True
    print('Prophet já disponível no ambiente.')
except Exception:
    print('Prophet não encontrado. Tentando instalar (pode demorar)...')
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'prophet', '-q'])
        from prophet import Prophet
        prophet_ok = True
        print('Prophet instalado com sucesso.')
    except Exception as e:
        print('Não foi possível instalar Prophet nesta sessão.')
        print('Detalhe:', e)
        print('Siga com ETS/ARIMA/ARIMAX normalmente.')


In [ ]:
# RUN_ME 27 (opcional) - Prophet com regressoras selic e evento
pred_prophet = None
mae_prophet = None
rmse_prophet = None

if prophet_ok:
    train_prophet = train_ts[['ds', 'y', 'selic', 'evento']].copy()
    test_prophet = test_ts[['ds', 'selic', 'evento']].copy()

    model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    model_prophet.add_regressor('selic')
    model_prophet.add_regressor('evento')
    model_prophet.fit(train_prophet)

    forecast = model_prophet.predict(test_prophet)
    pred_prophet = forecast['yhat'].values

    mae_prophet, rmse_prophet = calc_metrics(y_test_ts, pred_prophet)
    print('Prophet - MAE :', round(mae_prophet, 3))
    print('Prophet - RMSE:', round(rmse_prophet, 3))
else:
    print('Prophet indisponível nesta execução.')


In [ ]:
# RUN_ME 28 - tabela comparativa final (PARTE C)
results_ts = [
    {'modelo': 'Baseline_Naive', 'mae': mae_naive_ts, 'rmse': rmse_naive_ts},
    {'modelo': 'ETS_HoltWinters', 'mae': mae_ets, 'rmse': rmse_ets},
    {'modelo': 'ARIMA', 'mae': mae_arima, 'rmse': rmse_arima},
    {'modelo': 'ARIMAX_selic', 'mae': mae_arimax, 'rmse': rmse_arimax},
]

if pred_prophet is not None:
    results_ts.append({'modelo': 'Prophet', 'mae': mae_prophet, 'rmse': rmse_prophet})

results_ts_df = pd.DataFrame(results_ts).sort_values('rmse').reset_index(drop=True)
results_ts_df


In [ ]:
# RUN_ME 29 - gráfico real vs modelos (PARTE C)
plt.figure(figsize=(11, 4))
plt.plot(test_ts['ds'], y_test_ts, label='Real', color='black', linewidth=2)
plt.plot(test_ts['ds'], pred_naive_ts, label='Baseline_Naive', linewidth=1.8)
plt.plot(test_ts['ds'], pred_arimax, label='ARIMAX_selic', linewidth=1.8)
plt.plot(test_ts['ds'], pred_ets, label='ETS_HoltWinters', linewidth=1.8)

if pred_prophet is not None:
    plt.plot(test_ts['ds'], pred_prophet, label='Prophet', linewidth=1.8)

plt.title('PARTE C - Real vs previsões (teste)')
plt.xlabel('Data')
plt.ylabel('y (R$ milhões)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


## 5) Conclusão final

### Quando usar X -> y (regressão)
- Quando você tem drivers explicativos relevantes (macro, comercial, mix, risco).
- Bom para simular cenários e impacto de decisões (ex.: marketing, spread, mix).

### Quando usar t -> y (série temporal)
- Quando o histórico da própria série já carrega padrão forte (tendência/sazonalidade).
- Útil quando drivers são escassos, instáveis ou pouco confiáveis.

### Ponto crítico para ambos
- Sempre validar com **split temporal** e benchmark contra **baseline naive**.


## 6) Checklist FP&A

- Baseline: sempre comparar modelo com um baseline simples antes de “vender” ganho.
- Split temporal: nunca embaralhar dados de tempo ao validar forecast.
- Métricas: acompanhar MAE e RMSE no teste e monitorar estabilidade ao longo do tempo.
- Governança: documentar versão do modelo, features usadas e data de treino.
- Monitoramento: reavaliar performance periodicamente e re-treinar quando houver drift.
